# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new_dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature_engineering.html) --> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión.

4. [Submission Framework](https://fdelmazo.github.io/7506-Datos/TP2/submission_framework.html) --> Pequeño framework para construir las postulaciones de labels. 

5. [Parameter Tuning](https://fdelmazo.github.io/7506-Datos/TP2/parameter_tuning.html) --> Busqueda de los mejores hiper-parametros para cada algoritmo de ML.

6. [Feature Selection](https://fdelmazo.github.io/7506-Datos/TP2/feature_selection.html) --> Busqueda de la combinación de features más favorable.

7. TP2 (este notebook)--> Teniendo todo en cuenta, usando los dataframes con todos los atributos buscados y encontrados, se definen y aplican los algoritmos de clasificación, se realizan los entrenamientos y posteriores predicciones de conversiones y finalmente se arman las postulaciones de labels.

**Antes de comenzar, setear las credenciales (usuario y token)**
1. Visitar: https://www.kaggle.com/datatouille2018/account (con la cuenta que sea)
2. Tocar en Create New API Token
3. Guardar el archivo descargado en ~/.kaggle/kaggle.json

In [ ]:
# !unzip -q data/events_up_to_01062018.zip -d data

# !pip install kaggle
# !pip install nbimporter
# !conda install -y -c conda-forge xgboost 
# !conda install -y -c conda-forge lightgbm 
# !conda install -y -c conda-forge catboost

In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
import numpy as np
import time
import calendar
from itertools import combinations
import random
from time import sleep
from parameter_tuning import get_hiper_params
from feature_selection import get_feature_selection
import submission_framework as SF

seed = 42
hiper_params = get_hiper_params()
feature_selection = get_feature_selection()

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

display(df_users.head(), df_y.head())

## Algoritmos de Machine Learning

In [ ]:
posibilidades_algoritmos = []

---

### Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

model_name = 'decision_tree'
params = hiper_params[model_name]
model = DecisionTreeClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_name = 'random_forest'
params = hiper_params[model_name]
model = RandomForestClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### XGBoost


In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

model_name = 'xgboost'
params = hiper_params[model_name]
model = xgb.XGBClassifier(**params,random_state=seed)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression

model_name = 'logistic_regresion'
model = LogisticRegression(solver='lbfgs')
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_name = 'knn'
params = hiper_params[model_name]
K = params['n_neighbors']
model_name = f'KNN{K}'

model = KNeighborsClassifier(**params)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### Naive-Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB,ComplementNB

model_name = 'naive_bayes_Gaussian'
model = GaussianNB(**{'var_smoothing': 1e-09})
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)
posibilidades_algoritmos.append(model_with_name)

model_name = 'naive_bayes_Bernoulli'
model = BernoulliNB()
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)
posibilidades_algoritmos.append(model_with_name)

model_name = 'naive_bayes_Multinomial'
model = MultinomialNB()
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)
posibilidades_algoritmos.append(model_with_name)

model_name = 'naive_bayes_Complement'
model = ComplementNB()
model_with_name = (model_name,model)
SF.full_framework_wrapper(df_users,df_y,model_with_name)
posibilidades_algoritmos.append(model_with_name)

---

### LightGBM

In [ ]:
import lightgbm as lgb  #conda install -c conda-forge lightgbm 

model_name = 'lightgbm'
params = hiper_params[model_name]
model = lgb.LGBMClassifier(**params)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)

posibilidades_algoritmos.append(model_with_name)

---

### Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

model_name = 'neuralnetwork'
params = hiper_params[model_name]
model = MLPClassifier(**params)
model_with_name = (model_name, model)

# Funciona sólo con datos normalizados
SF.full_framework_wrapper(df_users, df_y, model_with_name, normalize=True)
posibilidades_algoritmos.append(model_with_name)

---

### Catboost

In [ ]:
# import catboost as cb #conda install -c conda-forge catboost

# model_name = 'catboost'
# params = hiper_params[model_name]

# model = cb.CatBoostClassifier(**params,verbose=False)
# model_with_name = (model_name,model)

# SF.full_framework_wrapper(df_users,df_y,model_with_name)
# posibilidades_algoritmos.append(model_with_name)

---

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as GBC  

model_name = 'gradient_boosting'
params = hiper_params[model_name]

model = GBC(**params)
model_with_name = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,model_with_name)
posibilidades_algoritmos.append(model_with_name)

---

## Encontrando el mejor submit

Corremos todos los algoritmos definidos sobre esas combinaciones, incluso ensamblados, en busqueda de su mejor combinación de hiper-parametros.

Finalmente, se corren todos los algoritmos en su mejor combinación contra todos los set de features definidos, en busqueda de la mejor fusión universal.

### Todos los posibles algoritmos

In [ ]:
print([x[0] for x in posibilidades_algoritmos])

In [ ]:
from sklearn.ensemble import BaggingClassifier

EXCLUDED = ['catboost','neuralnetwork',f'KNN{K}']

def bagging(posibilidades):
    posibilidades = list(filter(lambda x:x[0] not in EXCLUDED,posibilidades)) # Tarda muchísimo un ensamble con catboost
    baggins = [] # Frodo
    for n,m in posibilidades:
        baggins.append((n+'_bagging',BaggingClassifier(m)))
    return baggins

posibilidades_algoritmos_y_ensambles = posibilidades_algoritmos + bagging(posibilidades_algoritmos)

In [ ]:
EXCLUDED = ['catboost','neuralnetwork']

def ensamblar_algoritmos(posibilidades, n):
    posibilidades = list(filter(lambda x:x[0] not in EXCLUDED,posibilidades)) # Tarda muchísimo un ensamble con catboost
    result = list(combinations(posibilidades,n))
    result_names = [f'{x[0][0]}+{x[1][0]}' for x in result]
    return list(zip(result_names,result))

posibilidades_algoritmos_y_ensambles += ensamblar_algoritmos(posibilidades_algoritmos,2)

In [ ]:
def ensamble_tres_a_mano(nombre1,nombre2,nombre3):
    result = list(combinations(posibilidades_algoritmos, 3))
    for r in result:
        names = [x[0] for x in r]
        if nombre1 in names and nombre2 in names and nombre3 in names:
            result_names = '+'.join(names)
            return (result_names,r)
        
#posibilidades_algoritmos_y_ensambles += [ensamble_tres_a_mano()]

In [ ]:
print([x[0] for x in posibilidades_algoritmos_y_ensambles])

### Todos los posibles sets de features

In [ ]:
posibilidades_features = {
#     'Cumulative Importance':feature_selection['best_features_progresivo'],
#     'Forward Selection':feature_selection['best_features_forward'],
    'Backward Elimination':feature_selection['best_features_backward'],
#     'Stepwise Regression ':feature_selection['best_features_stepwise'],
#     'Full Dataframe':[],
}

In [ ]:
todo_junto = [x for f in posibilidades_features.values() for x in f]
intersec = list(set([x for x in todo_junto if todo_junto.count(x)>=2]))
# posibilidades_features['Feature Intersection'] = intersec

In [ ]:
# posibilidades_features['Seleccion a Mano'] = ['total_checkouts_month_5',
#                     'timestamp_last_checkout',
#                     'timestamp_last_event',
#                     'has_checkout_month_5',
#                     'total_checkouts',
#                     'days_to_last_event',
#                     'total_checkouts_last_week',
#                     'total_checkouts_months_1_to_4',
#                     'total_conversions',
#                     'total_session_conversion',
#                     'total_events',
#                     'total_sessions',
#                     'avg_events_per_session',
#                     'total_session_checkout',
#                     'has_checkout'
#                     ]

# posibilidades_features['Seleccion a Mano 2'] = ['dow_last_conversion', 
#                      'has_conversion_last_week', 'total_conversions_month_4', 
#                      'total_session_checkout', 'doy_last_conversion', 'timestamp_last_event', 
#                      'dow_last_checkout', 'total_checkouts', 'has_checkout', 'doy_last_checkout', 
#                      'has_checkout_month_1', 'timestamp_last_checkout', 'total_sessions', 
#                      'woy_last_event', 'has_checkout_month_5', 'avg_events_per_session']

In [ ]:
# cant_features = 30

# posibilidades_features[f'{cant_features} Random Sample'] = random.sample(df_users.columns.tolist(),cant_features)
# posibilidades_features[f'{cant_features} Random Sample 2'] = random.sample(df_users.columns.tolist(),cant_features)

In [ ]:
print([x for x in posibilidades_features])

---

### Combinando ambas ideas

In [ ]:
resultados = []
global_time = 0

for forma, features in posibilidades_features.items():
    global_start = time.process_time()
    print("\nTardó: {: ^100}s\n{: ^100s}".format(round(global_time,2),'-----------------------'))
    print(f'{forma}:\n')
    print(f'{features}\n\n')
    for nombre,algoritmo in posibilidades_algoritmos_y_ensambles:
        print('\t * ',end='')
        model_with_name = (f'{nombre}',algoritmo)
        start = time.process_time()
        model, auc = SF.full_framework_wrapper(df_users, df_y, model_with_name, columns=features)
        end = time.process_time()
        print(f'\t\t Tardó: {end-start:.2f}s')
        resultados.append((auc, forma, (nombre, algoritmo), features))
    global_end = time.process_time()
    global_start = global_end-global_start

In [ ]:
resultados.sort(reverse=True)
display([(x[0],x[1],x[2][0]) for x in resultados])

## Corrida Final

Se corre entrenando con X (y no X_train) el submit final.

In [ ]:
max_auc, campeon_forma, (campeon_nombre, campeon_algoritmo), campeon_features = resultados[1]
display(f"Mejor Apuesta: {campeon_nombre} ({max_auc:.4f} AUC) - Features: {campeon_forma}")
display(f"Features: {campeon_features}")

In [ ]:
print(f"{campeon_nombre} - {campeon_forma} - {max_auc:.4f}")
campeon_model, campeon_auc, csv_name, campeon_message = SF.full_framework_wrapper(df_users, 
                                                                                    df_y, 
                                                                                    (campeon_nombre,campeon_algoritmo),
                                                                                    columns=campeon_features,
                                                                                    submit=True,
                                                                                    all_in=True)   

#!kaggle competitions submit -f {csv_name} -m "{campeon_message}" trocafone

In [ ]:
# # Quemar n submits de punta a punta 

# for resultado in (resultados[1],resultados[2],resultados[5],resultados[6]):
#     print(f"\n\n{resultado[2][0]} - {resultado[1]} - {resultado[0]:.4f}\n\n)
#     max_auc, campeon_forma, (campeon_nombre, campeon_algoritmo), campeon_features = resultado
#     campeon_model, campeon_auc, csv_name, campeon_message = SF.full_framework_wrapper(df_users, 
#                                                                                     df_y, 
#                                                                                     (campeon_nombre,campeon_algoritmo),
#                                                                                     columns=campeon_features,
#                                                                                     submit=True,
#                                                                                     all_in=True)   
#     !kaggle competitions submit -f {csv_name} -m "{campeon_message}" trocafone
#     sleep(10)
#     print()

In [ ]:
#!kaggle competitions leaderboard -d trocafone
#!unzip -o trocafone.zip
#print('Last Best Score')
#!cat trocafone-publicleaderboard.csv | grep Datatouille | tail -n 1 | awk '{split($0,a,","); print "\t Fecha: " a[3] ; print "\t Porcentaje: " a[4]}'

https://www.kaggle.com/c/trocafone/submissions?sortBy=date

https://www.kaggle.com/c/trocafone/leaderboard